# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# File to Load (Remember to Change These)
file_to_load = "clean_city_data.csv"

# Read Purchasing File and store into Pandas data frame
clean_city_data = pd.read_csv(file_to_load)

#Display the summary data frame
clean_city_data.head()


,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Bethel,41.37,-73.41,296.48,88,75,3.10,US,1591915860
1,1,Goderich,43.75,-81.72,293.71,62,0,7.15,CA,1591916401
2,2,Hilo,19.73,-155.09,301.15,61,75,5.10,US,1591915700
3,3,Topolobampo,25.60,-109.05,307.59,52,5,7.70,MX,1591916402
4,4,Provideniya,64.38,-173.30,286.79,51,51,2.13,RU,1591915966


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure Gmaps
gmaps.configure(g_key)

In [4]:
# Use the Lat and Lng as locations and Humidity as the weight.
locations = clean_city_data[["Lat", "Lng"]].astype(float)

In [5]:
# Initialize (Heat)map
fig = gmaps.figure()

gmap_layer = gmaps.heatmap_layer(
    locations, 
    weights=clean_city_data["Humidity"],
    dissipating=False,
    max_intensity=100,
    point_radius = 1
)
fig.add_layer(gmap_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
vacation_city = city_data[(city_data['Max Temp'] > 298) 
                          & (city_data['Max Temp'] < 303) 
                          &(city_data['Cloudiness'] < 5)
                         &(city_data['Humidity'] <50)]
vacation_city

NameError: name 'city_data' is not defined

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
# Store into hotel_df
hotel_df= pd.DataFrame(vacation_city)

In [ ]:
hotel_df['Hotel Name'] = "NaN"
hotel_df.head()

In [ ]:
# Set parameters to search for hotels within 5000 meters
target_radius = 5000
target_type = "hotel"
target_search = "hotel"

In [ ]:
# Hit the Google Places API for each city's coordinates

hotel_df.loc[:,"Coordinates"] = hotel_df.loc[:,"Lat"].astype(str) +"," + hotel_df.loc[:,"Lng"].astype(str)
hotel_list = []

for location in hotel_df["Coordinates"]:
    params = {
        "location": location,
        "keyword": target_search,
        "radius": target_radius,
        "type": target_type,
        "key": g_key
    }

    gmaps_json = requests.get("https://maps.googleapis.com/maps/api/place/nearbysearch/json", params=params).json()
    
    try:
        hotel_list.append(gmaps_json["results"][0]["name"])
        
    except:
        hotel_list.append("N/A")
        print(f"No hotels within 5km of {location}.")

hotel_df["Hotel Name"] = hotel_list
hotel_df.loc[:, ~hotel_df.columns.str.contains('^Unnamed')]

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map

new_fig = fig = gmaps.figure()

markers = gmaps.marker_layer(locations, hotel_info)
new_fig.add_layer(gmap_layer)
new_fig.add_layer(markers)

# Display figure
new_fig